In [1]:
#! python2
# -*- coding: utf-8 -*-
import pprint
import pymongo
import datetime
import numpy as np
from pymongo import MongoClient

from scipy import spatial

import pylab

import os
8
import progressbar

db_name = 'twitter'
col_name = 'after_process'

In [2]:
from sklearn import metrics
from sklearn.cluster import DBSCAN
# from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity

from igraph import *
import igraph
import math
from operator import itemgetter



In [3]:
class MyPrettyPrinter(pprint.PrettyPrinter):
    def format(self, object, context, maxlevels, level):
        if isinstance(object, unicode):
            return (object.encode('thai'), True, False)
        return pprint.PrettyPrinter.format(self, object, context, maxlevels, level)

def go_print( input ):
    MyPrettyPrinter().pprint(input)
    # ppp = pprint.PrettyPrinter(indent=4)
    # ppp.pprint(input)
    return;

def get_midnight(time):
    return time.replace(minute=0, hour=0, second=0, microsecond=0)

def get_thai_midnight(time):
    out = time + datetime.timedelta(hours=7)
    out = out.replace(minute=0, hour=0, second=0, microsecond=0) - datetime.timedelta(hours=7)
    return out

In [4]:
if __name__ == '__main__':
    client = MongoClient()
    db = client[db_name]

    result = db[col_name].create_index([('ts', pymongo.ASCENDING)])
    cursor = db[col_name].find({ 'ts':{'$gte':1511654400000} }).sort([('ts', pymongo.ASCENDING)]).limit(40000)
    

    date = []
    text = []
    text_2 = []
    all_doc = []

    bar = progressbar.ProgressBar(maxval=cursor.count()+1, widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    data=1
    for doc in cursor:
        data += 1
        bar.update(data)
        
        ts = doc['ts']
        datetime_object = datetime.datetime.fromtimestamp(ts/1000)
        if True:
#         if len(doc['hashtags']) != 0:
            buf = doc['nouns']
            buf_2 = doc['nouns_nltk']
            for x in doc['hashtags']:
                if not x.lower() in buf:
                    buf.append(x.lower())
                if not x.lower() in buf_2:
                    buf_2.append(x.lower())
            all_doc.append(doc)
            date.append(datetime_object)
            text.append(' '.join(buf))
            text_2.append(' '.join(buf_2))
    bar.finish()
#     print len(date)

[########################################################################] 100%


In [5]:
def two_str(num):
    if num < 10:
        return str(0)+str(num)
    else:
        return str(num)

In [6]:
#time_gap <= 60
def split_time_gap(date,text,all_doc,time_gap):
    s_text = {}
    s_all_doc = {}
    for datetime_object in date:
        i = date.index(datetime_object)
        m = datetime_object.minute
        h = datetime_object.hour
        d = datetime_object.day
        key = two_str(d)+'/'+two_str(datetime_object.month)+'::'+two_str(h)+'.'+two_str((m/time_gap)*time_gap)
        if not s_text.has_key(key):
            s_text[key] = []
            s_all_doc[key] = []
        s_text[key].append(text[i])
        s_all_doc[key].append(all_doc[i])
    return s_text,s_all_doc
s_text,s_all_doc = split_time_gap(date,text,all_doc,60)
s_text_2,s_all_doc = split_time_gap(date,text_2,all_doc,60)

In [7]:
from math import *
import operator as op
import itertools

L = dict() # Lattice
Lkey = dict() # Key of each itemset
X = 0
N = 0

for c in range(100):
    L[c] = dict()
    Lkey[c] = dict()

In [8]:
#  n เลือก r ไม่สนลำดับ
def nCr(n, r):
    if n < r: return 0
    r = min(r, n-r)
    if r == 0: return 1
    numer = reduce(op.mul, range(n, n-r, -1))
    denom = reduce(op.mul, range(1, r+1))
    return numer//denom

In [9]:
def streamMining(stream, theta, epsilon):
    global L, Lkey
    for c in range(100):
        L[c] = dict()
        Lkey[c] = dict()
    X = 0
    N = 0
    print("Run streamMining")
    T = list() # Buffer
    f = 0 # Average 2-itemsets per transaction
    c = 0 # Number of call -> reducFreq
    for t in stream:
        T.append(t)
        update(t, 1)
        update(t, 2)
        f = twoItemsetPerTransaction(t)
        if len(L[2]) >= ceil(1/(theta*epsilon)) * f:
            reducFreq(2)
            c += 1
            i = 2
            while len(L[i]) != 0:
                i += 1
                for tp in T:
                    if len(tp) >= i:
                        update(tp, i)
                reducFreq(i)
            for ii in range(i,100):
                L[ii] = dict()
                Lkey[ii] = dict()
            T = list()
    for k,v in L.items():
        sorted_v = sorted(v.items(), key=operator.itemgetter(1), reverse=True)
        L[k] = sorted_v
    return L

In [10]:
def twoItemsetPerTransaction(t):
    global X, N
    f = 0
    N += 1
    X += nCr(len(t), 2)
    f = ceil(X/N)
    return f

In [11]:
def update(t, i):
    global L, Lkey
    itemsets = createItemsets(t, i)

    for s in itemsets:
        if len(set(s)) != i:
            continue
        if s in L[i]:
            L[i][s] += 1
        elif i <= 2:
            L[i][s] = 1
        elif i > 2:
            mini_itemsets = createItemsets(s, i-1)
            chk = True
            for mini in mini_itemsets:
                if mini not in L[i-1]:
                    chk = False
                    break
            if chk:
                L[i][s] = 1

In [12]:
def reducFreq(i):
    global L
    remove = list()
    for s in L[i]:
        L[i][s] -= 1
        if L[i][s] <= 0:
            remove.append(s)
    for r in remove:
        del L[i][r]

In [13]:
def createItemsets(t, i):
    return tuple(map( lambda w: tuple(sorted(w)), list(itertools.combinations(t, i)) ) )

In [14]:
output= {}
for k,v in s_text.items():
    sample = v
    sample_doc = s_all_doc[k]

    x = streamMining(map( lambda w: w.split(' '), sample ),0.01,0.01)
    
    output[k] = x.copy()

Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining


In [15]:
#     output_text 
#     output_nouns 
#     out_nmf 
#     out_lda 
for k,v in sorted(output.items()):
    print '------' ,k, '-----'
    try:
        
        for data in v.values():
            remain = 15
            for top in data:
                print top
                remain -= 1
                if remain == 0:
                    break
    except :
        pass
    

------ 26/11::07.00 -----
(('',), 204)
((u'day',), 53)
((u'game',), 51)
((u'job',), 50)
((u'time',), 49)
((u'people',), 45)
((u'team',), 44)
((u'night',), 40)
((u'today',), 37)
((u'hiring',), 37)
((u'tonight',), 33)
((u'morning',), 32)
((u'thing',), 31)
((u'alabama',), 28)
((u'auburn',), 26)
((u'hiring', u'job'), 37)
((u'careerarc', u'job'), 20)
((u'careerarc', u'hiring'), 20)
((u'job', u'opening'), 16)
((u'hiring', u'opening'), 16)
((u'people', u'time'), 13)
((u'eagle', u'war'), 12)
((u'job', u'team'), 10)
((u'mph', u'wind'), 10)
((u'careerarc', u'opening'), 9)
((u'college', u'football'), 9)
((u'opening', u'team'), 9)
((u'hiring', u'team'), 9)
((u'spring', u'valley'), 8)
((u'spring', u'toll'), 8)
------ 26/11::08.00 -----
(('',), 290)
((u'time',), 57)
((u'man',), 43)
((u'people',), 38)
((u'day',), 37)
((u'year',), 36)
((u'job',), 34)
((u'girl',), 30)
((u'game',), 28)
((u'amp',), 27)
((u'today',), 27)
((u'night',), 26)
((u'thing',), 25)
((u'christma',), 24)
((u'tonight',), 24)
((u'hiri

In [16]:
output_2= {}
for k,v in s_text_2.items():
    sample = v
    sample_doc = s_all_doc[k]

    x = streamMining(map( lambda w: w.split(' '), sample ),0.01,0.01)
    
    output_2[k] = x.copy()

Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining
Run streamMining


In [17]:
#     output_text 
#     output_nouns 
#     out_nmf 
#     out_lda 
for k,v in sorted(output_2.items()):
    print '------' ,k, '-----'
    try:
        for data in v.values():
            remain = 15
            for top in data:
                print top
                remain -= 1
                if remain == 0:
                    break
    except :
        pass
    

------ 26/11::07.00 -----
(('',), 102)
((u'time',), 53)
((u'day',), 53)
((u'game',), 51)
((u'job',), 50)
((u'peopl',), 45)
((u'team',), 44)
((u'night',), 41)
((u'today',), 37)
((u'hiring',), 37)
((u'thing',), 31)
((u'morn',), 31)
((u'love',), 30)
((u'don',), 29)
((u'amp',), 28)
((u'hiring', u'job'), 37)
((u'careerarc', u'hiring'), 20)
((u'careerarc', u'job'), 20)
((u'gt', u'humid'), 13)
((u'gt', u'wind'), 13)
((u'peopl', u'time'), 13)
((u'job', u'team'), 10)
((u'tvpersonality2017', u'vote'), 10)
((u'gt', u'temperatur'), 10)
((u'karanvohra', u'vote'), 10)
((u'eagl', u'war'), 10)
((u'vote', u'zindagikimehak'), 10)
((u'job', u'recommend'), 10)
((u'job', u'open'), 9)
((u'colleg', u'footbal'), 9)
------ 26/11::08.00 -----
(('',), 146)
((u'time',), 57)
((u'peopl',), 38)
((u'man',), 37)
((u'day',), 37)
((u'year',), 36)
((u'job',), 34)
((u'amp',), 34)
((u'game',), 29)
((u'lol',), 28)
((u'night',), 28)
((u'girl',), 28)
((u'today',), 27)
((u'thing',), 25)
((u'love',), 25)
((u'hiring', u'job'), 2